In [1]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

from synthetic_data.tasks.writing import _get_gutenberg_subset

gutenberg_pq: pl.DataFrame = pl.read_parquet("../dataset_files/gutenberg_backtranslate_from_txt_conversations.parquet")

In [3]:
gutenberg_pq.columns

['conversation']

In [4]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from IPython.display import Markdown
from synthetic_data.tasks.writing import GutenbergBacktranslationFromTxt

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

first_sample = gutenberg_pq.sample(n=1)
task = GutenbergBacktranslationFromTxt()
total = 0

for row in gutenberg_pq.sample(10).iter_rows(named=True):
    print(row.keys())
    display(Markdown(f"**{row['title']}**"))
    row_out = await task.preprocess_row(row)
    for row in row_out:
        n_tokens = sum([len(tiktoken_encoder.encode(par)) for par in row['text']])
        print("-"*100)
        display(Markdown(row['text']))
        print(f"n_tokens: {n_tokens}")
        total += 1
    if total > 10:
        break


dict_keys(['conversation'])


KeyError: 'title'

In [ ]:
import re

input = first_sample.to_dicts()
formatted_out = task.format_input_conversation(ldictl(input))

In [ ]:
formatted_out[0]